In [ ]:
pip install pandasql

In [105]:

import pandas as pd
from pandasql import sqldf
df=pd.read_csv('penguins.csv')
print(df.shape)
print(df['species'].unique())
df.head()



(342, 5)
['Adelie' 'Gentoo' 'Chinstrap']


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181,3750
1,Adelie,39.5,17.4,186,3800
2,Adelie,40.3,18.0,195,3250
3,Adelie,36.7,19.3,193,3450
4,Adelie,39.3,20.6,190,3650


# *A Quick Recap*

SQL is a mostly standardized langauge used to access databases, however, there are some differences between implentations. For this implmentation, we will use a pandas wrapper for SQLite

## Basic Syntax


An SQL statment is structured around the the **SELECT** and **FROM** command and mirrors the following structure:

SELECT < function < column> >,...,< function< column > > FROM < < Database > >

The spaces are due to markdown issues not acutal code

In [5]:
new=sqldf(

"SELECT * FROM df"
)
new.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181,3750
1,Adelie,39.5,17.4,186,3800
2,Adelie,40.3,18.0,195,3250
3,Adelie,36.7,19.3,193,3450
4,Adelie,39.3,20.6,190,3650


In [8]:
new=sqldf(

"SELECT bill_length_mm FROM df"
)
new.head()

,bill_length_mm
0,39.1
1,39.5
2,40.3
3,36.7
4,39.3


## **Filtering**

Adding a **WHERE** clause to the end of a query allows us to filter the return data. It syntax is as follows:

SELECT < function < column> >,...,< function< column > > FROM < < Database > > WHERE < conditional >

In [73]:
new=sqldf(

"SELECT bill_length_mm FROM df WHERE (bill_length_mm like '5%')"
)
new.head()

,bill_length_mm
0,50.0
1,50.0
2,50.2
3,50.0
4,59.6


### Quick Side Note

In [74]:
new=sqldf(

"SELECT bill_length_mm FROM df WHERE (bill_length_mm GLOB '5*')"
)
new.head()

,bill_length_mm
0,50.0
1,50.0
2,50.2
3,50.0
4,59.6


In [75]:
%%timeit -r 10 -n 100
new=sqldf(

"SELECT bill_length_mm FROM df WHERE (bill_length_mm like '5%')"
)

19.2 ms ± 492 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [76]:
%%timeit -r 10 -n 100
new=sqldf(

"SELECT bill_length_mm FROM df WHERE (bill_length_mm GLOB '5*')"
)

19.1 ms ± 540 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


### **Aggragating and Grouping**
Adding a **GROUP BY** clause to the end of a query allows us to find information based on specific attributes. It syntax is as follows:
SELECT < function < column> >,...,< function< column > > FROM < < Database > > WHERE < conditional > GROUP BY < column >


In [106]:
new=sqldf(

"SELECT bill_length_mm, COUNT(*) FROM df WHERE bill_length_mm >= 35 GROUP BY bill_length_mm"
)
new.head()

,bill_length_mm,COUNT(*)
0,35.0,2
1,35.1,1
2,35.2,1
3,35.3,1
4,35.5,2


### **Nesting Queries**

A Nested Query is the process of injecting a query into another and is equivilent to calling a function within a function. Let's take a look!

Say we want to know which species has the largest bill and what that size is,the python would look like this:

In [100]:
maxval=df['bill_length_mm'].max()
new=(df.loc[    
    df['bill_length_mm'] == maxval]
    .iloc[:,0:2])
new.head()

,species,bill_length_mm
184,Gentoo,59.6


In [42]:
new=sqldf(

"""
SELECT species,bill_length_mm  FROM df WHERE bill_length_mm = 
    (SELECT MAX(bill_length_mm) FROM df)
"""

)
new.head()

,species,bill_length_mm
0,Gentoo,59.6


In [40]:
new=sqldf(

"""SELECT species, MAX(big) FROM (SELECT species, MAX(bill_length_mm) AS big FROM df GROUP BY species)"""

)
new.head()

,species,MAX(big)
0,Gentoo,59.6


In [44]:
new=sqldf(

"""SELECT species, bill_length_mm FROM df WHERE species=
    (SELECT species FROM df WHERE bill_length_mm = 
        (SELECT MAX(bill_length_mm) FROM df))"""

)
new.head()

,species,bill_length_mm
0,Gentoo,46.1
1,Gentoo,50.0
2,Gentoo,48.7
3,Gentoo,50.0
4,Gentoo,47.6


Importantly there is a limit

### But wait, what is the time complexity?

In [64]:
%%timeit -r 10 -n 100
new=sqldf(

"""SELECT species,bill_length_mm  FROM df WHERE bill_length_mm = 
    (SELECT MAX(bill_length_mm) FROM df)"""

)



19.2 ms ± 514 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [65]:
%%timeit -r 10 -n 100
new=sqldf(

"""SELECT species, MAX(big) FROM (SELECT species, MAX(bill_length_mm) AS big FROM df GROUP BY species)"""

)

19.5 ms ± 670 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


### **Is this efficent?**


Question:
How would you find all information for the penguine with largest body mass that has an even leading digit?
Hint the or opperator is the same as it was in regex ie []

In [93]:
new=sqldf(
"""

SELECT *, MAX(body_mass_g) AS mass FROM (SELECT * FROM df where body_mass_g glob '[2468]*')
"""
)
new.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,mass
0,Gentoo,49.2,15.2,221,6300,6300
